# Drem
We'd like to identify clusters in the WG data and see how these clusters differ between young, old and IR. First we format the data drem-wise

## Do imports

In [2]:
import os
import pandas
import matplotlib.pyplot as plt
import seaborn
import glob
import numpy

## Get Data

In [3]:
dire = r'/home/b3053674/Documents/WaferGen/2D_TGFb/Drem'
os.chdir(dire)
data_dir = os.path.join(dire,'normalized_data.xlsx')

data = pandas.read_excel(data_dir,header= [0,1], index_col=[0,1])
##swap index levels
data = data.swaplevel(0,1)
data

Repeat                     1                                              \
Time                      0          1          2          4          8    
Group    Gene                                                              
Adult    ABL1       0.378423   0.248551   0.179178   0.150038   0.334396   
IR       ABL1       0.108694   0.133757   0.078691   0.085270   0.069162   
Neonatal ABL1       0.148102   0.124111   0.128679   0.070827   0.100453   
Adult    ACTA2      0.041949   0.029644   0.038330   0.055325   0.054825   
IR       ACTA2      0.098117   0.072037   0.079774   0.093161   0.085593   
Neonatal ACTA2      0.126545   0.094145   0.125453   0.134191   0.108241   
Adult    ACTB_ECM   0.713688   0.751430   0.692461   0.602025   0.685952   
IR       ACTB_ECM   0.940949   0.788284   0.872353   0.716799   1.037774   
Neonatal ACTB_ECM   0.693160   0.689003   0.696296   0.693923   0.872453   
Adult    ACTB_NC    0.856921   0.867283   0.891803   0.828433   0.962194   
IR       ACTB_NC    1.209876   0.989733   0.992868   0.942172   1.418822   
Neonatal ACTB_NC    1.018011   0.981337   0.930575   1.195667   1.173398   
Adult    B2M_ECM    1.048273   0.939673   1.056611   1.168563   1.120490   
IR       B2M_ECM    1.195678   1.361585   1.228982   1.530593   1.116817   
Neonatal B2M_ECM    1.163442   1.199349   1.173968   1.186003   0.936369   
Adult    B2M_NC     0.981761   1.013028   0.925303   0.987558   1.017959   
IR       B2M_NC     1.000339   1.174259   1.136135   1.031363   1.073397   
Neonatal B2M_NC     1.043227   1.109839   1.025519   0.933904   0.880969   
Adult    BGN        0.054223   0.058091   0.060682   0.071635   0.090975   
IR       BGN        0.131757   0.144149   0.113115   0.150722   0.170964   
Neonatal BGN        0.183359   0.225765   0.237715   0.347785   0.345146   
Adult    CAV1       1.411387   1.507841   1.558856   1.330674   1.394356   
IR       CAV1       1.320312   1.061814   1.068009   1.148692   0.752292   
Neonatal CAV1       1.031536   0.933299   1.112885   0.954678   0.736860   
Adult    CDC42      0.133012   0.143710   0.141678   0.120409   0.105495   
IR       CDC42      0.125347   0.119708   0.114162   0.139564   0.101419   
Neonatal CDC42      0.145424   0.138268   0.154476   0.133229   0.146221   
Adult    CDKN1A     0.276656   0.289642   0.248419   0.205091   0.177998   
IR       CDKN1A     0.955084   0.926300   0.783613   0.785380   0.795489   
Neonatal CDKN1A     0.367268   0.448491   0.448672   0.183810   0.319450   
...                      ...        ...        ...        ...        ...   
Adult    THBS1      0.424923   0.518940   0.542049   0.319380   0.433520   
IR       THBS1      2.220383   2.816214   2.421757   1.942158   2.139395   
Neonatal THBS1      2.004802   2.561930   2.758862   1.990205   1.813993   
Adult    THBS2      0.205579   0.241528   0.216601   0.248391   0.263561   
IR       THBS2      0.050742   0.054441   0.067834   0.076037   0.063098   
Neonatal THBS2      0.126295   0.151278   0.145871   0.146294   0.157611   
Adult    TIMP1      2.043574   1.808170   1.866485   1.838011   1.807643   
IR       TIMP1      2.993242   3.052215   3.011546   2.947333   2.860551   
Neonatal TIMP1      2.977971   2.591938   3.511489   3.536562   2.724419   
Adult    TIMP2      2.780212   2.552534   2.134613   2.786561   2.631326   
IR       TIMP2      1.929422   1.900747   1.787898   1.871429   1.773099   
Neonatal TIMP2      2.167482   2.230043   2.075779   2.282694   1.954770   
Adult    TIMP3      0.447216   0.399327   0.460026   0.495170   0.551349   
IR       TIMP3      0.202006   0.208856   0.202826   0.221125   0.192310   
Neonatal TIMP3      0.053425   0.050907   0.057607   0.071802   0.067575   
Adult    TNF             NaN        NaN        NaN        NaN        NaN   
IR       TNF             NaN        NaN        NaN        NaN   0.000912   
Neonatal TNF             NaN        NaN        NaN        NaN        NaN   
Adult    VCAN       0.129417   0.128855   0.0990

The following block of code writes a function to format WG data for drem input and write to file.

# Note: ALSO need to computer log2 ratios

I began writing some code to normalize the data to the 0 time point but actually I'm not convinced this is necessary. If we had treated Vs control for the WG data then it'll be useful to take log2 ratios of treated Vs control but since we only have a 0 time point control and Drem actually does this transformation for us anyay, this is just not needed. Bear in mind though for future data sets. 

In [82]:
def compute_log2_ratio(data):
    """
    Computer log2 ratio for each time point vs 0 time point for WG data
    
    args:
        data. Normalized Wafergen data file
    """
    df_dct = {}
    for label, df in data.groupby(level=[0,1], axis=1):
        df_dct[label] = df[label]/data[(1,0)]
    df =pandas.DataFrame( df_dct)

        
    print df
    
print compute_log2_ratio(data)

                     1                                                    \
                    0         1         2         4         8         12   
Group    Gene                                                              
Adult    ABL1      1.0  0.656807  0.473486  0.396481  0.883655  0.470727   
IR       ABL1      1.0  1.230591  0.723972  0.784499  0.636298  0.753264   
Neonatal ABL1      1.0  0.838009  0.868851  0.478232  0.678269  0.721477   
Adult    ACTA2     1.0  0.706666  0.913745  1.318871  1.306969  0.846072   
IR       ACTA2     1.0  0.734198  0.813044  0.949489  0.872353  1.194981   
Neonatal ACTA2     1.0  0.743965  0.991373  1.060417  0.855351  1.348378   
Adult    ACTB_ECM  1.0  1.052883  0.970257  0.843542  0.961138  1.132142   
IR       ACTB_ECM  1.0  0.837754  0.927099  0.761783  1.102902  1.072460   
Neonatal ACTB_ECM  1.0  0.994003  1.004525  1.001101  1.258660  1.929299   
Adult    ACTB_NC   1.0  1.012092  1.040707  0.966756  1.122850  1.094888   
IR       ACT

In [ ]:

os.chdir(dire)

def drem_format(data, group = 'Adult'):
    """
    Format data for drem and output to folder
    """
    ## Ceheck group exists
    if group not in list(set(data.index.get_level_values(0))):
        raise Exception('group parameter does not exist')
        
    folder = os.path.join(os.getcwd(), group)
    ## create dir if not exist
    if os.path.isdir(folder)!=True:
        os.mkdir(folder)
    
    ##reserve original directory for afterwards
    cwd = os.getcwd()
    ## cd to dir
    os.chdir(folder)
    
    ## group by group index
    for label, df in data.groupby(level = 0):
        if label == group:
            df.index = df.index.droplevel(0)
            df.index.name = 'Symbol'
            
            ## write each repeat to file
            for repeat in list(set(df.columns.get_level_values(0))):
                df[repeat].to_csv('{}_{}.csv'.format(group,repeat), sep='\t')
    ## ensure return to original directory
    os.chdir(cwd)
    return 0
        
groups = ['Adult', 'Neonatal', 'IR']
[drem_format(data, group) for group in groups] 

# Drem configurations
Drem can be used in batch mode by specifying a configuration file for the settings and an output file for the model. This can be used to systematically test of the effects of parameters of a model but also to run the same settings per WG data set. 

In [61]:


def write_config(input_file, output_file, kwarg_dct={}):
    """
    Write a Drem config file. This will be useful for drem analysis using same settings per data set. 
    Also should be able to more easily run batch mode
    
    args:
        input_file:
            drem configuration file to start from
            
        output_file:
            name of output drem configuration file
    kwargs:
        kwarg_dct:
            default={}. Keys are option names, values are new values for option. 
    
    
    """
    dire = os.path.dirname(input_file)
    ## get information from default filename
    filename = os.path.join(dire, input_file)
    if os.path.isfile(filename)!=True:
        raise Exception('{} is not a file'.format(filename))
    
    ##read
    with open(filename) as f:
        default = f.read()
    
    ##parse to pandas
    df = pandas.DataFrame(default.split('\n'))
    option, value = [], []
    dct = {}
    
    ## split by tab
    for row in range(df.shape[0]):
        split = df.iloc[row][0].split('\t')
        if len(split)==2 and split[0][0]!='#':
            dct[split[0]] = split[1]
    
    ## label the results df
    df = pandas.DataFrame(dct, index=[0]).transpose()
    df.index.name = 'Option'
    df.columns = ['Value']
    
    
    expression_file = pandas.DataFrame({'Option': 'Expression_Data_File',
                                        'Value' : ''}, index=[0])
    expression_file = expression_file.set_index('Option')
    df = pandas.concat([df, expression_file], join='inner',axis=0)
    df = df.sort_index()
    
    for kwarg in kwarg_dct:
        if kwarg not in list(df.index):
            raise Exception('{} not in available options. These are your options: {}'.format(kwarg, pandas.DataFrame(df.index)))
        
        ## update dict
        df.loc[kwarg] = kwarg_dct[kwarg]
        
    ## write to csv
    df.to_csv(output_file, sep='\t', header = None)
    print 'writing to {}'.format(output_file)
    return df
        

    


## write sh file
Here we create the drem files that we want to run via batch mode, create a shell script and run it

In [62]:


def configure_wg_drem(pattern, input_file, output_file):
    """

    """
    files =sorted(glob.glob(pattern))
    
    
    if files==[]:
        raise Exception('pattern did not match. ')
    repeat_string =''
    
    kwarg_dct = {'Repeat_Data_Files(comma delimited list)': ','.join(files[1:]),
         'Expression_Data_File': files[0],
         'Allow_Path_Merges': 'false',
         'Convergence_Likelihood_%': 0.01,
         'Filter_Gene_If_It_Has_No_Static_Input_Data': 'false',
         'Key_Input_X_p-val_10^-X' : 1.5,
         'Only_include_annotations_with_these_evidence_codes': 9696,
         'Penalized_Likelihood_Node_Penalty':0.01,
          }
    
    write_config(input_file, output_file, kwarg_dct)
    
    
    
    


patterns = [os.path.join(os.getcwd(),'Adult/Adult_*'),
            os.path.join(os.getcwd(),'IR/IR_*'),
            os.path.join(os.getcwd(),'Neonatal/Neonatal_*')]

outputs = [os.path.join(os.getcwd(), 'AdultConfig.txt'),
           os.path.join(os.getcwd(), 'IRConfig.txt'),
           os.path.join(os.getcwd(), 'NeonatalConfig.txt')]

input_file = os.path.join(os.getcwd(),'DefaultConfigFile.txt')

print [configure_wg_drem(patterns[i], input_file, outputs[i]) for i in range(len(patterns))]


writing to /home/b3053674/Documents/WaferGen/2D_TGFb/Drem/AdultConfig.txt
writing to /home/b3053674/Documents/WaferGen/2D_TGFb/Drem/IRConfig.txt
writing to /home/b3053674/Documents/WaferGen/2D_TGFb/Drem/NeonatalConfig.txt
[None, None, None]


In [66]:
def foo(bar):
    return bar